<h1 align="center">Текстовый калькулятор  правильных, неправильных и смешанных дробей  и обычных чисел с математическим порядком вычисления</h1>

<h2 align="center"> Кизяков Дмитрий ИТиАБД ПМ20-6 </h2>

<h3>Инструкция по эксплуатации</h3>

<ol>
  <li>Знаки арифметических действий записываются следующим образом<ul>
    <li>" + " - сложение</li>
    <li>" - " - вычитание</li>
    <li>" * " - умножение</li>
    <li>" / " - деление</li>
    <li>" ^ " - возведение в степень</li>
    
  </ul>  
  </li>
  <li>Способ записи смешанных дробей – 5(1/4), правильных – 5/9, неправильных 19/7</li>
  <li>Для обозначения отрицательного числа/дроби минус ставится непоследственно перед самим числом/дробью</li>
  <li>Перед каждым знаком действий и после него необходимо ставить пробел</li>
  <li>До и после скобок не следует ставить пробелов, если не протеворечит п.4</li>
  <li>Возведение возможно только в натуральные степени для сохранения концепции работы с рациональными числами</li>
  <li>Действия с результатом выражения и команды для их реализации<ul>
    <li>"exit" - завершение работы</li>
    <li>"save ___" - сохранение результата в указанную переменную</li>
    <li>"to double" - преведение ответа к десятичному виду</li>
    <li>"to mixed" - приведение ответа к смеженой дроби, если возможно</li>

  </ul>  
  </li>
  <li>Имя переменной, в которую сохраняется результат операции должен содержать не менее одной латинской буквы любого регистра</li>
</ol>

In [42]:
import re

def brackets_check(string):
    count = 0
    for i in string:
        if i =="(":
            count += 1
        elif i == ")":
             count -= 1
    return bool(count)
        

def simplification(x):
    if x[0] == 0:
        return [0, 1]
    a = abs(x[0])
    b = abs(x[1])
    while a != b:
        if a > b:
            a -= b
        else:
            b -= a
    if x[0]*x[1] < 0:
        return [-abs(int(x[0]/a)), abs(int(x[1]/a))]
    else:
        return [int(x[0]/a),int(x[1]/a)]
    
def mixed_fraction(whole_part, fraction):
    if whole_part[0] == "-":
        return simplification([int(whole_part[:-1]) * fraction[1] - fraction[0], fraction[1]])
    else:
        return simplification([int(whole_part[:-1]) * fraction[1] + fraction[0], fraction[1]])

fraction = lambda x: simplification([int(x.split("/")[0]), int(x.split("/")[1])])

integer = lambda x: [int(x), 1]
    
multiplication = lambda x, y: simplification([x[0] * y[0], x[1] * y[1]])

division = lambda x, y: simplification([x[0] * y[1], x[1] * y[0]])

addition = lambda x, y: simplification([x[0] * y[1] + x[1] * y[0], x[1] * y[1]])
    
subtruction = lambda x, y: simplification([x[0] * y[1] - x[1] * y[0], x[1] * y[1]])

power = lambda x, n: simplification([x[0] ** n[0], x[1] ** n[0]])

def to_mixed(lst):
    if abs(lst[0]) < lst[1]:
        return str(lst[0]) + "/" + str(lst[1])
    if str(lst[0])[0] == "-":
        return "-" + str(abs(lst[0]) // lst[1]) + "(" + str(abs(lst[0]) % lst[1]) + "/" + str(lst[1]) + ")"
    else:
        return str(abs(lst[0]) // lst[1]) + "(" + str(abs(lst[0]) % lst[1]) + "/" + str(lst[1]) + ")"

to_double = lambda x: x[0] / x[1]

to_fraction = lambda x: x[0] if x[1] == 1 else str(x[0]) + "/" + str(x[1])

def string_remastering(expression_string):
    lst = re.sub('\)', ' )', re.sub('\(', '( ', expression_string)).split()
    p = re.compile(r"[\d]+[/][\d]+")
    lst2 = [fraction(i) if p.match(i) else i for i in lst]
    lst2 = ["ничего"] + ["ничего"] + lst2 + ["ничего"]
    p1 = re.compile(r"[-]?[\d]+[/(]")
    lst3 = []
    for i in range (2, len(lst2) - 1):
        if type(lst2[i]) == str:
            if lst2[i] == ")" and type(lst2[i - 2]) == str:
                if p1.match(lst2[i - 2]):
                    continue
                else:
                    lst3.append(lst2[i])
            elif lst2[i].isdigit():
                lst3.append(integer(int(lst2[i])))
            elif p1.match(lst2[i]) and lst2[i + 1]:
                lst3.append(mixed_fraction(lst2[i], lst2[i+1]))
            else:
                lst3.append(lst2[i])
        
        if type(lst2[i]) == list:
            if p1.match(lst2[i - 1]):
                continue
            else:
                lst3.append(lst2[i])
    return lst3

def str_counter(lst, variables):
    
    if "Деление на ноль" in lst:
        return "Деление на ноль"
    
    while ")" in lst:
        for i, el in enumerate(lst):
            if el == "(":
                left_ = i
            if el == ")":
                right_ = i
                break
        lst = lst[:left_] + str_counter(lst[left_+1:right_], variables) + lst[right_+1:]
        
    if "Деление на ноль" in lst:
        return "Деление на ноль"
        
            
    p = re.compile(r"[\w]*[a-zA-Z][\w]*")
    for i in range (len(lst)):
        if type(lst[i]) == str:
            if p.match(lst[i]):
                if lst[i] in variables:
                    lst[i] = variables[lst[i]]
                else:
                    raise KeyError('Данная переменная не сохранена')
    
    lst = ["ничего"] + lst + ["ничего"]
    lst2 = []
    for i in range (1,len(lst)-1):
        if type(lst[i]) == list:
            if lst[i-1] != "^":
                lst2.append(lst[i])
            else: 
                continue
        
        elif type(lst[i]) == str:
            if lst[i] == "^":
                lst2.append(power(lst2[-1], lst[i+1]))
                lst2.pop(-2)
            else: 
                lst2.append(lst[i])
    
    lst = ["ничего"] + lst2 + ["ничего"]
    lst2 = []
    for i in range (1,len(lst)-1):
        if type(lst[i]) == list:
            if lst[i-1] not in "*/":
                lst2.append(lst[i])
            else: 
                continue
        
        elif type(lst[i]) == str:
            if lst[i] == "*":
                lst2.append(multiplication(lst2[-1], lst[i+1]))
                lst2.pop(-2)
            elif lst[i] == "/":
                if lst[i+1][0] == 0:
                    return ["Деление на ноль"]
                lst2.append(division(lst2[-1], lst[i+1]))
                lst2.pop(-2)
            else: 
                lst2.append(lst[i])
    
    lst = ["ничего"] + lst2 + ["ничего"]
    lst2 = []
    for i in range (1,len(lst)-1):
        if type(lst[i]) == list:
            if lst[i-1] not in "+-":
                lst2.append(lst[i])
            else: 
                continue
        
        elif type(lst[i]) == str:
            if lst[i] == "+":
                lst2.append(addition(lst2[-1], lst[i+1]))
                lst2.pop(-2)
            elif lst[i] == "-":
                lst2.append(subtruction(lst2[-1], lst[i+1]))
                lst2.pop(-2)
            else: 
                lst2.append(lst[i])
    return lst2
    
    
def calculator(string, variables):
    return str_counter(string_remastering(string), variables)






dic = {}
last = None
while True:
    string = input()
    if string == "exit":
        print("Работа завершена")
        break
    elif string.split()[0] == "save":
        if last:
            dic[string.split()[1]] = last
            print("Сохранено успешно")
        else:
            print("Для записи необходимо произвести хотя бы одно вычисление")
    elif string.split()[0] == "del":
        if string.split()[1] in dic:
            dic.pop(string.split()[1])
            print("Удалено успешно")
        else:
            print("Данная переменная не является сохранённой")
    elif string == "to double":
        if last:
            print(to_double(last))
        else:
            print("Для преобразования необходимо произвести хотя бы одно вычисление")
    elif string == "to mixed":
        if last:
            print(to_mixed(last))
        else:
            print("Для преобразования необходимо произвести хотя бы одно вычисление")
    else:
        if brackets_check(string):
            print("Неправильно введены скобки")
            continue
        resault = calculator(string, dic)
        if "Деление на ноль" in resault:
            print("Деление на ноль")
            continue
        resault = resault[0]
        last = resault
        print(to_fraction(resault))

6/4 / 4/9 + (5/4 * (6/7))
249/56
5 + (9/7 - (13/27 * 52/79)) - 5 * 63/8
-3990305/119448
((5(6/7) * 5 - 3/13) + 3(6/97)) * 6/53
1700970/467831
exit
Работа завершена
